In [1]:
%matplotlib inline
from collections import Counter
from typing import Callable, List, Dict
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import functions as F
import pyspark as ps
import json
import requests
import scipy.stats as stats

# Always make it pretty.
plt.style.use('ggplot')

In [2]:
# Create spark session
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('julia_json')
         .getOrCreate()
        )
sc = spark.sparkContext

In [4]:
spark_df = spark.read.json('data/french_tweets.json')
spark_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [5]:
spark_df.show(5)

+------------+-----------+--------------------+------------------+--------------------+-----------------+--------------+--------------+---------+------------+----+------------------+------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+--------------------+------------------+-------------+----------------+--------------------+-------------+---------+--------------------+--------------------+-------------+---------+--------------------+
|contributors|coordinates|          created_at|display_text_range|            entities|extended_entities|extended_tweet|favorite_count|favorited|filter_level| geo|                id|            id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|limit|               place|possibly_sensitive|quoted_status|quoted_status_id|quoted_status_id_str|retweet_count|re